# hey, databse! Code Tutorial

## Initial Setup

In [1]:
import sys
import os
from pathlib import Path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('hey-database-tutorial')

# credentials if nedeed
api_key = os.getenv('OPENAI_API_KEY')
db_pwd = os.getenv('POSTGRES_PWD')

from src.llm_handler.openai_handler import OpenAIHandler
llm = OpenAIHandler(
    api_key=api_key,
    chat_model="gpt-4o"
)

## Test LLM

In [7]:
system_prompt = "Sei un simpatico standup comedian"
user_prompt = "Raccontami una breve storia simpatica. Molto breve."

response = llm.get_completion(
    prompt=user_prompt,
    system_prompt=system_prompt,
    temperature=0.2
)

print("\nRisposta LLM:")
print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Risposta LLM:
Certo! Allora, c'era una volta un pinguino che voleva diventare un detective. Si mise un cappello da investigatore e una lente d'ingrandimento, ma ogni volta che cercava di seguire una pista... scivolava! Alla fine decise di specializzarsi nei casi di ghiaccio sottile.


# DB connection

In [3]:
from src.connettori.postgres import PostgresManager # also supported mysql, vertica and snowflake

# create connection
db = PostgresManager(
    host="localhost",
    port="5432",
    database="giacomo",
    user="postgres",
    password=db_pwd
)

# check connection
if db.connect():
    print("Connessione stabilita con successo!")
    
    # Test query
    result = db.execute_query("SELECT COUNT(*) FROM northwind.customers")
    if result:
        columns, data = result
        print(f"Numero totale ordini: {data[0][0]}")
else:
    print("Errore nella connessione!")

Connessione stabilita con successo!
Numero totale ordini: 91


## Metadata extraction

In [ ]:
from src.schema_metadata.postgres_metadata_retriever import PostgresMetadataRetriever
from src.schema_metadata.enhancement_strategy import MetadataEnhancementStrategy

# We can decide if we want to enhance the metadata retrieved from the database
# enhancing mean use an LLM to add more information to the metadata, like table description
class SimpleEnhancementStrategy(MetadataEnhancementStrategy):
    def should_enhance(self) -> bool:
        return False # True to enhance metadata, False to keep the original metadata

# create metadata retriever
metadata_retriever = PostgresMetadataRetriever(
    db_engine=db.engine,
    llm_handler=llm,
    enhancement_strategy=SimpleEnhancementStrategy(),
    schema="northwind"  # your schema here
)

metadata = metadata_retriever.get_all_tables_metadata()

print(metadata)

INFO:hey-database:Inizializzando metadata retriever per schema: northwind
INFO:hey-database:Loading metadata from database
INFO:hey-database:Estraggo i metadati per la tabella: territories
INFO:hey-database:Estraggo i metadati per la tabella: order_details
INFO:hey-database:Estraggo i metadati per la tabella: employee_territories
INFO:hey-database:Estraggo i metadati per la tabella: us_states
INFO:hey-database:Estraggo i metadati per la tabella: customers
INFO:hey-database:Estraggo i metadati per la tabella: orders
INFO:hey-database:Estraggo i metadati per la tabella: employees
INFO:hey-database:Estraggo i metadati per la tabella: shippers
INFO:hey-database:Estraggo i metadati per la tabella: products
INFO:hey-database:Estraggo i metadati per la tabella: categories
INFO:hey-database:Estraggo i metadati per la tabella: suppliers
INFO:hey-database:Estraggo i metadati per la tabella: region
INFO:hey-database:Estraggo i metadati per la tabella: customer_demographics
INFO:hey-database:Estra

{'territories': EnhancedTableMetadata(base_metadata=TableMetadata(name='territories', columns=[{'name': 'territory_id', 'type': 'VARCHAR(20)', 'nullable': False, 'distinct_values': ['90405', '30346', '48084', '95054', '20852', '03049', '72716', '01730', '75234', '07960', '32859', '02184', '80909', '48075', '27403', '95008', '40222', '31406', '08837', '19428', '94105', '02116', '01833', '01581', '10019', '94025', '85014', '55439', '55113', '48304', '60179', '98052', '98004', '53404', '10038', '03801', '85251', '14450', '95060', '02903', '78759', '11747', '27511', '06897', '29202', '19713', '98104', '44122', '02139', '60601', '45839', '80202', '33607']}, {'name': 'territory_description', 'type': 'VARCHAR(60)', 'nullable': False, 'distinct_values': ['Neward', 'Santa Cruz', 'Columbia', 'New York', 'Westboro', 'Philadelphia', 'Phoenix', 'San Francisco', 'Dallas', 'Roseville', 'Southfield', 'Austin', 'Rockville', 'Morristown', 'Orlando', 'Portsmouth', 'Georgetow', 'Santa Clara', 'Chicago', '

Right now access to the data is not immediate, partly because we have to go through the dataclas (see cell below).  
One might think about how to make accessing the info more intuitive, more dictionary style.

In [5]:
metadata['territories'].base_metadata.foreign_keys

[{'constrained_columns': ['region_id'],
  'referred_table': 'region',
  'referred_columns': ['region_id']}]

## Store

In [6]:
from src.embedding.openai_embedding import OpenAIEmbedding
from src.store.qdrant_vectorstore import QdrantStore

# embedding model for vectorize documents in store
embedding_model = OpenAIEmbedding(
    api_key=api_key,
    model="text-embedding-3-small"
)

# store initialization
vector_store = QdrantStore(
    path="./data/tutorial",  # per storage locale
    collection_name="tutorial_collection",
    embedding_model=embedding_model
)

# collection initialization
if vector_store.initialize():
    print("Vector store inizializzato con successo!")
    
    # Popola con i metadati
    if vector_store.populate_store_with_metadata(metadata):
        print("Metadati caricati nel vector store!")
        
        # Prova una ricerca
        user_query = "Trova informazioni sugli ordini e i clienti"
        results = vector_store.search_similar_tables(user_query, limit=2)
        
        print("\nTabelle più rilevanti per la query:")
        for result in results:
            print(f"\nTabella: {result.table_name}")
            print(f"Score: {result.relevance_score}")
            print(f"Descrizione: {result.metadata.description}")

INFO:hey-database:Collection tutorial_collection already exists
INFO:hey-database:Collection already populated, skipping metadata population


Vector store inizializzato con successo!
Metadati caricati nel vector store!


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"



Tabelle più rilevanti per la query:

Tabella: orders
Score: 0.3863829238671133
Descrizione: 

Tabella: customers
Score: 0.3790589369388381
Descrizione: 


## SQL Agent

In [8]:
from src.config.languages import SupportedLanguage
from src.agents.sql_agent import SQLAgent
from dataclasses import dataclass

# prompt config
@dataclass
class SimplePromptConfig:
    include_sample_data: bool = False # include or not table rows in prompt
    max_sample_rows: int = 3

# sql agent initialization
sql_agent = SQLAgent(
    db=db,
    llm_manager=llm,
    metadata_retriever=metadata_retriever,
    schema_name="northwind",  # schema here
    prompt_config=SimplePromptConfig(),
    vector_store=vector_store,
    language=SupportedLanguage.ITALIAN # da semplificare
)

# Prova una query in linguaggio naturale
question = "Quali sono i 5 clienti che hanno fatto più ordini?"
response = sql_agent.run(question)

if response.success:
    print("\nQuery SQL generata:")
    print(response.query)
    print("\nSpiegazione:")
    print(response.explanation)
    if response.results:
        print("\nRisultati:")
        for row in response.results[:3]:  # primi 3 risultati
            print(row)
else:
    print(f"\nErrore: {response.error}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Query SQL generata:
SELECT c.customer_id, c.company_name, COUNT(o.order_id) AS total_orders FROM northwind.customers c JOIN northwind.orders o ON c.customer_id = o.customer_id GROUP BY c.customer_id, c.company_name ORDER BY total_orders DESC LIMIT 5;

Spiegazione:
La query seleziona i 5 clienti che hanno effettuato il maggior numero di ordini. Utilizza una JOIN tra le tabelle 'customers' e 'orders' per contare il numero di ordini per ciascun cliente. I risultati sono ordinati in ordine decrescente in base al numero totale di ordini e limitati ai primi 5 clienti. Verranno restituiti l'ID del cliente, il nome dell'azienda e il numero totale di ordini effettuati.

Risultati:
{'customer_id': 'SAVEA', 'company_name': 'Save-a-lot Markets', 'total_orders': 31}
{'customer_id': 'ERNSH', 'company_name': 'Ernst Handel', 'total_orders': 30}
{'customer_id': 'QUICK', 'company_name': 'QUICK-Stop', 'total_orders': 28}
